In [1]:
import matplotlib.pyplot as plt
import keras
from keras.datasets import cifar10,fashion_mnist
import os
from scipy import misc

import numpy as np
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D,LeakyReLU,Conv2DTranspose, Conv2D
from keras.optimizers import Adam
import os
from keras.layers.convolutional import _Conv
from keras.legacy import interfaces
from keras.engine import InputSpec
import cv2
import keras.backend.tensorflow_backend as KTF

from scipy import misc
def set_gpu_config(device = "0",fraction=0.25):
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = fraction
    config.gpu_options.visible_device_list = device
    KTF.set_session(tf.Session(config=config))


def predict_images(file_name, generator, noise_size, n = 10, size = 32):

    image = generator.predict(np.random.normal(size=(n*n, ) + noise_size))

    image = np.reshape(image, (n, n, size, size, 3))
    image = np.transpose(image, (0, 2, 1, 3, 4))
    image = np.reshape(image, (n*size, n*size, 3))

    image = 255 * (image + 1) / 2
    image = image.astype("uint8")
    misc.imsave(file_name, image)
def build_generator(input_shape):
    model = Sequential()

    model.add(Conv2DTranspose(256,(3,3),strides=(2,2),padding="same",input_shape=input_shape))
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(128,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(64,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(32,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(1,(3,3),padding="same",activation="tanh"))
    model.summary()
    return model


def build_discriminator(input_shape):
    model = Sequential()

    model.add(SNConv2D(32,(3,3),strides=(2,2),padding="same",input_shape=input_shape))
    model.add(LeakyReLU(0.2))

    model.add(SNConv2D(64,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(SNConv2D(128,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(SNConv2D(256,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(SNConv2D(1,(3,3),padding="same"))
    model.add(GlobalAveragePooling2D())
    model.summary()

    return model

def build_functions(batch_size, noise_size, image_size, generator, discriminator):

    noise = K.random_normal((batch_size,) + noise_size,0.0,1.0,"float32")
    real_image = K.placeholder((batch_size,) + image_size)
    fake_image = generator(noise)

    d_input = K.concatenate([real_image, fake_image], axis=0)
    pred_real, pred_fake = tf.split(discriminator(d_input), num_or_size_splits = 2, axis = 0)

    d_loss = K.mean(K.maximum(0., 1 - pred_real)) + K.mean(K.maximum(0., 1 + pred_fake))
    g_loss = -K.mean(pred_fake)

    d_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(d_loss, discriminator.trainable_weights)
    d_train = K.function([real_image, K.learning_phase()], [d_loss], d_training_updates)

    g_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(g_loss, generator.trainable_weights)
    g_train = K.function([real_image, K.learning_phase()], [g_loss], g_training_updates)

    return d_train,g_train

class SNConv2D(_Conv):
    @interfaces.legacy_conv2d_support
    def __init__(self, filters,
                 kernel_size,
                 strides=(1, 1),
                 padding='valid',
                 data_format=None,
                 dilation_rate=(1, 1),
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):

        super(SNConv2D, self).__init__(
            rank=2,
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            padding=padding,
            data_format=data_format,
            dilation_rate=dilation_rate,
            activation=activation,
            use_bias=use_bias,
            kernel_initializer=kernel_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
            activity_regularizer=activity_regularizer,
            kernel_constraint=kernel_constraint,
            bias_constraint=bias_constraint,
            **kwargs)

        self.input_spec = InputSpec(ndim=4)
        self.Ip = 1
        self.u = self.add_weight(
            name='W_u',
            shape=(1,filters),
            initializer='random_uniform',
            trainable=False
        )

    def call(self, inputs):
        outputs = K.conv2d(
            inputs,
            self.W_bar(),
            strides=self.strides,
            padding=self.padding,
            data_format=self.data_format,
            dilation_rate=self.dilation_rate)

        if self.use_bias:
            outputs = K.bias_add(
                outputs,
                self.bias,
                data_format=self.data_format)

        if self.activation is not None:
            return self.activation(outputs)
        return outputs


    def get_config(self):
        config = super(SNConv2D, self).get_config()
        config.pop('rank')
        return config

    def W_bar(self):
        # Spectrally Normalized Weight
        W_mat = K.permute_dimensions(self.kernel, (3, 2, 0, 1)) # (h, w, i, o) => (o, i, h, w)
        W_mat = K.reshape(W_mat,[K.shape(W_mat)[0], -1]) # (o, i * h * w)

        if not self.Ip >= 1:
            raise ValueError("The number of power iterations should be positive integer")

        _u = self.u
        _v = None

        for _ in range(self.Ip):
            _v = _l2normalize(K.dot(_u, W_mat))
            _u = _l2normalize(K.dot(_v, K.transpose(W_mat)))

        sigma = K.sum(K.dot(_u,W_mat)*_v)

        K.update(self.u,K.in_train_phase(_u, self.u))
        return self.kernel / sigma

def _l2normalize(x):
    return x / K.sqrt(K.sum(K.square(x)) + K.epsilon())
set_gpu_config("0",0.5)

epochs = 50
image_size = (32,32,1)
noise_size = (2,2,32)
batch_size = 64
sample_size=10
size=32
sample_interval=200
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train=[cv2.resize(i,(32,32)) for i in x_train]
x_test=[cv2.resize(i,(32,32)) for i in x_test]
x_train=np.array(x_train)
x_test=np.array(x_test)
x_train=np.expand_dims(x_train,axis=3)
x_test=np.expand_dims(x_test,axis=3)
num_of_data = x_train.shape[0]
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train = (x_train/255)*2-1
x_test = (x_test/255)*2-1
y_train = keras.utils.to_categorical(y_train,10)
y_test = keras.utils.to_categorical(y_test,10)

generator = build_generator(noise_size)
discriminator = build_discriminator(image_size)
d_train, g_train = build_functions(batch_size, noise_size, image_size, generator, discriminator)

nb_batches = int(x_train.shape[0] / batch_size)
global_step = 0
for epoch in range(epochs):
    for index in range(nb_batches):
        global_step += 1
        real_images = x_train[index * batch_size:(index + 1) * batch_size]
        d_loss, = d_train([real_images, 1])
        g_loss, = g_train([real_images, 1])
        print("[{0}/{1}] [{2}_{3}] d_loss: {4:.4}, g_loss: {5:.4}".format(epoch, epochs, epoch, global_step, d_loss,
                                                                              g_loss))
        if global_step % sample_interval == 0:
            image = generator.predict(np.random.normal(size=(sample_size * sample_size,) + noise_size))
            image = 255 * (image + 1) / 2
            # print(np.array(image).shape)
            # image = np.reshape(image, (sample_size, sample_size, size, size, 1))
            # image = np.transpose(image, (0, 2, 1, 3, 4))
            # image = np.reshape(image, (sample_size * size, sample_size * size, 1))
            #

            # image = image.astype("uint8")
            # image=np.squeeze(image)
            # if not os.path.isdir('images_sngan_fashion_mnist'):
            #     os.mkdir('images_sngan_fashion_mnist')
            #
            # misc.imsave("images_sngan_fashion_mnist/e{0}_step{1}img.png".format(epoch,global_step), image)
            fig, axs = plt.subplots(sample_size, sample_size)
            cnt = 0
            for i in range(sample_size):
                for j in range(sample_size):
                    axs[i, j].imshow(image[cnt, :, :, 0], cmap='gray')
                    axs[i, j].axis('off')
                    cnt += 1
            if not os.path.isdir('images_sngan_fashion_mnist'):
                os.mkdir('images_sngan_fashion_mnist')
            fig.savefig("images_sngan_fashion_mnist/epoch_%d_step_%d.png" % (epoch, global_step))
            plt.close()


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose_1 (Conv2DTr (None, 4, 4, 512)         147968    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 4, 4, 512)         0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 8, 8, 256)         1179904   
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 16, 16, 128)       295040    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_transpose_4 (Conv2DTr (None, 32, 32, 64)        73792     
__________

[0/50] [0_121] d_loss: 0.03972, g_loss: 2.13
[0/50] [0_122] d_loss: 0.05747, g_loss: 1.366
[0/50] [0_123] d_loss: 0.01771, g_loss: 1.88
[0/50] [0_124] d_loss: 0.0895, g_loss: 0.9193
[0/50] [0_125] d_loss: 0.2569, g_loss: 5.945
[0/50] [0_126] d_loss: 1.896, g_loss: 0.83
[0/50] [0_127] d_loss: 0.2336, g_loss: 2.172
[0/50] [0_128] d_loss: 0.3565, g_loss: 1.195
[0/50] [0_129] d_loss: 0.06817, g_loss: 1.613
[0/50] [0_130] d_loss: 0.007103, g_loss: 1.39
[0/50] [0_131] d_loss: 0.004016, g_loss: 1.533
[0/50] [0_132] d_loss: 0.009792, g_loss: 1.454
[0/50] [0_133] d_loss: 0.02539, g_loss: 1.216
[0/50] [0_134] d_loss: 0.04179, g_loss: 3.042
[0/50] [0_135] d_loss: 0.4703, g_loss: 0.6659
[0/50] [0_136] d_loss: 0.6166, g_loss: 4.255
[0/50] [0_137] d_loss: 0.6871, g_loss: 1.772
[0/50] [0_138] d_loss: 0.09615, g_loss: 1.361
[0/50] [0_139] d_loss: 0.001236, g_loss: 1.254
[0/50] [0_140] d_loss: 0.03976, g_loss: 1.307
[0/50] [0_141] d_loss: 0.01249, g_loss: 1.554
[0/50] [0_142] d_loss: 0.06807, g_loss: 1

[0/50] [0_312] d_loss: 0.6849, g_loss: 0.4309
[0/50] [0_313] d_loss: 1.003, g_loss: 2.657
[0/50] [0_314] d_loss: 0.6943, g_loss: 1.292
[0/50] [0_315] d_loss: 0.5139, g_loss: 1.672
[0/50] [0_316] d_loss: 0.3504, g_loss: 1.522
[0/50] [0_317] d_loss: 0.4987, g_loss: 2.451
[0/50] [0_318] d_loss: 0.4647, g_loss: 0.6232
[0/50] [0_319] d_loss: 1.173, g_loss: 3.717
[0/50] [0_320] d_loss: 1.629, g_loss: 0.7826
[0/50] [0_321] d_loss: 0.6532, g_loss: 1.168
[0/50] [0_322] d_loss: 0.6577, g_loss: 1.443
[0/50] [0_323] d_loss: 0.8299, g_loss: 1.609
[0/50] [0_324] d_loss: 0.269, g_loss: 1.298
[0/50] [0_325] d_loss: 0.5173, g_loss: 1.733
[0/50] [0_326] d_loss: 0.4189, g_loss: 1.786
[0/50] [0_327] d_loss: 0.3007, g_loss: 1.892
[0/50] [0_328] d_loss: 0.5416, g_loss: 2.067
[0/50] [0_329] d_loss: 0.4255, g_loss: 2.615
[0/50] [0_330] d_loss: 0.4545, g_loss: 0.8015
[0/50] [0_331] d_loss: 0.746, g_loss: 3.633
[0/50] [0_332] d_loss: 1.02, g_loss: 0.6568
[0/50] [0_333] d_loss: 0.7584, g_loss: 1.741
[0/50] [0_33

[0/50] [0_503] d_loss: 0.85, g_loss: 0.7038
[0/50] [0_504] d_loss: 0.8811, g_loss: 1.774
[0/50] [0_505] d_loss: 0.8785, g_loss: 0.959
[0/50] [0_506] d_loss: 0.6212, g_loss: 2.103
[0/50] [0_507] d_loss: 0.7119, g_loss: 0.3473
[0/50] [0_508] d_loss: 1.081, g_loss: 2.17
[0/50] [0_509] d_loss: 0.8976, g_loss: 0.1892
[0/50] [0_510] d_loss: 0.9858, g_loss: 1.215
[0/50] [0_511] d_loss: 0.6544, g_loss: 0.9679
[0/50] [0_512] d_loss: 0.904, g_loss: 1.202
[0/50] [0_513] d_loss: 0.7992, g_loss: 0.8825
[0/50] [0_514] d_loss: 0.7626, g_loss: 1.873
[0/50] [0_515] d_loss: 0.6128, g_loss: 0.6532
[0/50] [0_516] d_loss: 0.7331, g_loss: 2.545
[0/50] [0_517] d_loss: 0.4293, g_loss: 0.2492
[0/50] [0_518] d_loss: 1.328, g_loss: 2.081
[0/50] [0_519] d_loss: 1.41, g_loss: 0.715
[0/50] [0_520] d_loss: 0.8582, g_loss: 1.114
[0/50] [0_521] d_loss: 0.916, g_loss: 1.655
[0/50] [0_522] d_loss: 0.6208, g_loss: 1.669
[0/50] [0_523] d_loss: 0.5913, g_loss: 2.382
[0/50] [0_524] d_loss: 0.4476, g_loss: 0.798
[0/50] [0_52

...

[49/50] [49_46142] d_loss: 0.0, g_loss: 2.174
[49/50] [49_46143] d_loss: 0.02451, g_loss: 1.968
[49/50] [49_46144] d_loss: 0.007244, g_loss: 2.63
[49/50] [49_46145] d_loss: 0.01857, g_loss: 2.813
[49/50] [49_46146] d_loss: 0.004681, g_loss: 2.47
[49/50] [49_46147] d_loss: 0.01036, g_loss: 2.986
[49/50] [49_46148] d_loss: 0.00321, g_loss: 2.022
[49/50] [49_46149] d_loss: 0.06888, g_loss: 3.826
[49/50] [49_46150] d_loss: 0.1371, g_loss: 1.456
[49/50] [49_46151] d_loss: 0.07788, g_loss: 1.248
[49/50] [49_46152] d_loss: 0.01663, g_loss: 2.146
[49/50] [49_46153] d_loss: 0.02286, g_loss: 2.018
[49/50] [49_46154] d_loss: 0.03135, g_loss: 2.014
[49/50] [49_46155] d_loss: 0.0, g_loss: 1.912
[49/50] [49_46156] d_loss: 0.01073, g_loss: 2.17
[49/50] [49_46157] d_loss: 0.02323, g_loss: 2.366
[49/50] [49_46158] d_loss: 0.01094, g_loss: 2.618
[49/50] [49_46159] d_loss: 0.05091, g_loss: 2.664
[49/50] [49_46160] d_loss: 0.0, g_loss: 2.468
[49/50] [49_46161] d_loss: 0.07118, g_loss: 2.883
[49/50] [49_46

[49/50] [49_46312] d_loss: 0.05142, g_loss: 1.422
[49/50] [49_46313] d_loss: 0.1067, g_loss: 0.981
[49/50] [49_46314] d_loss: 0.04674, g_loss: 2.051
[49/50] [49_46315] d_loss: 0.02379, g_loss: 2.728
[49/50] [49_46316] d_loss: 0.0, g_loss: 2.586
[49/50] [49_46317] d_loss: 0.03922, g_loss: 1.923
[49/50] [49_46318] d_loss: 0.0127, g_loss: 2.299
[49/50] [49_46319] d_loss: 0.0424, g_loss: 2.965
[49/50] [49_46320] d_loss: 0.06779, g_loss: 2.594
[49/50] [49_46321] d_loss: 0.04766, g_loss: 2.38
[49/50] [49_46322] d_loss: 0.08225, g_loss: 2.927
[49/50] [49_46323] d_loss: 0.09271, g_loss: 1.972
[49/50] [49_46324] d_loss: 0.02548, g_loss: 2.39
[49/50] [49_46325] d_loss: 0.0169, g_loss: 2.166
[49/50] [49_46326] d_loss: 0.1171, g_loss: 1.796
[49/50] [49_46327] d_loss: 0.08155, g_loss: 2.273
[49/50] [49_46328] d_loss: 0.0, g_loss: 2.369
[49/50] [49_46329] d_loss: 0.00407, g_loss: 2.587
[49/50] [49_46330] d_loss: 0.1167, g_loss: 2.214
[49/50] [49_46331] d_loss: 0.06422, g_loss: 1.787
[49/50] [49_4633

[49/50] [49_46481] d_loss: 0.1047, g_loss: 2.228
[49/50] [49_46482] d_loss: 0.06001, g_loss: 2.548
[49/50] [49_46483] d_loss: 0.00184, g_loss: 2.901
[49/50] [49_46484] d_loss: 0.06459, g_loss: 2.626
[49/50] [49_46485] d_loss: 0.03962, g_loss: 2.29
[49/50] [49_46486] d_loss: 0.006567, g_loss: 2.626
[49/50] [49_46487] d_loss: 0.0959, g_loss: 1.854
[49/50] [49_46488] d_loss: 0.04461, g_loss: 2.168
[49/50] [49_46489] d_loss: 0.07701, g_loss: 1.468
[49/50] [49_46490] d_loss: 0.07009, g_loss: 2.207
[49/50] [49_46491] d_loss: 0.118, g_loss: 1.532
[49/50] [49_46492] d_loss: 0.07185, g_loss: 1.904
[49/50] [49_46493] d_loss: 0.02566, g_loss: 1.837
[49/50] [49_46494] d_loss: 0.000432, g_loss: 2.117
[49/50] [49_46495] d_loss: 0.02821, g_loss: 2.61
[49/50] [49_46496] d_loss: 0.1103, g_loss: 1.686
[49/50] [49_46497] d_loss: 0.02692, g_loss: 2.345
[49/50] [49_46498] d_loss: 0.002304, g_loss: 2.652
[49/50] [49_46499] d_loss: 0.02063, g_loss: 2.375
[49/50] [49_46500] d_loss: 0.06028, g_loss: 1.933
[49/

[49/50] [49_46651] d_loss: 0.0184, g_loss: 2.877
[49/50] [49_46652] d_loss: 0.0216, g_loss: 2.583
[49/50] [49_46653] d_loss: 0.02524, g_loss: 2.189
[49/50] [49_46654] d_loss: 0.03101, g_loss: 2.209
[49/50] [49_46655] d_loss: 0.05228, g_loss: 3.397
[49/50] [49_46656] d_loss: 0.113, g_loss: 2.138
[49/50] [49_46657] d_loss: 0.06781, g_loss: 1.917
[49/50] [49_46658] d_loss: 0.03074, g_loss: 2.469
[49/50] [49_46659] d_loss: 0.04976, g_loss: 2.559
[49/50] [49_46660] d_loss: 0.1168, g_loss: 1.831
[49/50] [49_46661] d_loss: 0.1133, g_loss: 1.423
[49/50] [49_46662] d_loss: 0.03203, g_loss: 1.858
[49/50] [49_46663] d_loss: 0.0296, g_loss: 1.657
[49/50] [49_46664] d_loss: 0.06965, g_loss: 1.542
[49/50] [49_46665] d_loss: 0.06343, g_loss: 2.281
[49/50] [49_46666] d_loss: 0.07619, g_loss: 1.963
[49/50] [49_46667] d_loss: 0.01026, g_loss: 2.366
[49/50] [49_46668] d_loss: 0.05331, g_loss: 2.874
[49/50] [49_46669] d_loss: 0.05663, g_loss: 2.345
[49/50] [49_46670] d_loss: 0.0, g_loss: 2.365
[49/50] [49

[49/50] [49_46821] d_loss: 0.0, g_loss: 3.037
[49/50] [49_46822] d_loss: 0.07456, g_loss: 2.147
[49/50] [49_46823] d_loss: 0.02988, g_loss: 1.521
[49/50] [49_46824] d_loss: 0.0198, g_loss: 1.668
[49/50] [49_46825] d_loss: 0.003492, g_loss: 2.274
[49/50] [49_46826] d_loss: 0.01057, g_loss: 3.041
[49/50] [49_46827] d_loss: 0.02457, g_loss: 2.536
[49/50] [49_46828] d_loss: 0.02982, g_loss: 1.738
[49/50] [49_46829] d_loss: 0.001169, g_loss: 2.203
[49/50] [49_46830] d_loss: 0.0, g_loss: 2.102
[49/50] [49_46831] d_loss: 0.02894, g_loss: 2.47
[49/50] [49_46832] d_loss: 0.02463, g_loss: 1.633
[49/50] [49_46833] d_loss: 0.04529, g_loss: 1.428
[49/50] [49_46834] d_loss: 0.07571, g_loss: 1.206
[49/50] [49_46835] d_loss: 0.02057, g_loss: 1.9
[49/50] [49_46836] d_loss: 0.05747, g_loss: 2.217
[49/50] [49_46837] d_loss: 0.04283, g_loss: 2.644
[49/50] [49_46838] d_loss: 0.009237, g_loss: 3.754
[49/50] [49_46839] d_loss: 0.02333, g_loss: 2.92
[49/50] [49_46840] d_loss: 0.02235, g_loss: 2.538
[49/50] [4